In [ ]:
#Object detection using MNIST fashion dataset
#Reference:  https://youtu.be/bemDFpNooA8 

%load_ext tensorboard
import tensorflow as tf
from tensorflow import keras
import numpy as np
from datetime import datetime

print(tf.__version__)

#Load dataset splitting between 'train' and 'test' subsets (60K train, 10K test)
#Separate train/test to prevent bias decisions (use blind data to test)
#Train labels are numbers, not text to support multiple languages
fashionMNIST = keras.datasets.fashion_mnist
(trainImages, trainLabels), (testImages, testLabels) = fashionMNIST.load_data()

#Show values from the images (one of the articles of clothing)
import matplotlib.pyplot as plt
plt.imshow(trainImages[5])
print(trainLabels[5])
print(trainImages[5])

#Normalize the images so values range between 0..1
#Since each pixel is 8 bits we can do this by dividing by 255.0
trainImages = trainImages/255.0
testImages = testImages/255.0

#Setup logs for tensorboard -- didn't work here....
log_dir = "/tmp/logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#Define the Neural Network architecture (1 input layer, 1 hidden layer, 1 output layer)
#Input is the 28x28 images --> input_shape=(28,28) and we flatten to 784x1 vector
#Output is the number of categories --> 10 for the fashion_mnist dataset
#Units is 128 --> number of neurons in the hidden layer 
#Activation is a decision function that quantizes the output based on some rules
#   relu == rectified linear unit
#   softmax picks the 'best' number in a set based on probabilities
model = keras.Sequential([
      keras.layers.Flatten(input_shape=(28,28)),
      keras.layers.Dense(128, activation=tf.nn.relu),
      keras.layers.Dense(10, activation=tf.nn.softmax)
])
#Initialize model with random values and calculate loss function and accuracy
#model.compile(optimizer=tf.train.AdamOptimizer(),  tf.train.AdamOptimizer is no longer supported in TensorFlow v2
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


#Train the model with the training data then evaluate with the test data
model.fit(trainImages, trainLabels, epochs=50, callbacks=[tensorboard_callback])
testLoss, testAcc = model.evaluate(testImages, testLabels)


In [ ]:
#Predict for new images - use some random ones from the dataset
print('\n--------TEST with random images -------')

#Get some random test images
#The original load randomly split the images between train and test
myTestImages, myTestLabels = trainImages[0:50], trainLabels[0:50]
myTestImages = myTestImages/255.

#Make predictions from the trained model on the random input images
predictions = model.predict(myTestImages)

#Create 1 row with 4 figures
fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(16,8))
pltCnt = 0

#Show the image and its label then print the predicted label
from random import randint

for i in [randint(0, 49),randint(0, 49),randint(0, 49),randint(0, 49)]:
  ax[pltCnt].imshow(myTestImages[i]) 
  print('Test Image label: ', myTestLabels[i])
  print('Predicted label: ', np.amax(predictions[i]), '\n')
  pltCnt+=1

